<a id='top'></a>

# Méthodes de Monte-Carlo en Finance

# Projet 2 : Option barrière et diffusion tuée

Pierre ARTIGALA - Omar MANSOUR

## Consignes

Ce projet a pour but d'étudier le calcul du prix de certaines options barrières par méthode de Monte Carlo. Il se fonde sur l'article "Weak approximation of killed diffusion using Euler schemes" qui étudie deux méthodes de simulations. Votre rapport comportera au minimum :
- Une présentation du prix des options barrières dans le modèle de Black-Scholes pour obtenir des formules fermées.
- Une présentation des deux méthodes de l'article en commentant les démarches de preuves.
- Des simulatons testant ces méthodes (contre le formules fermées).

# Table of contents

## [Imports](#imports)
## [1. Prix des options barrières dans le modèle de Black-Scholes - Formules fermées](#part1)
## [2. Méthodes de Monte Carlo (article)](#part2)
## [3. Comparaison des différentes méthodes](#part3)

<a id='imports'></a>

# Imports [[^]](#top)

In [1]:
import numpy as np
from scipy.stats import norm

<a id='part1'></a>

# 1. Prix des options barrières dans le modèle de Black-Scholes - Formules fermées [[^]](#top)

In [104]:
def delta_plus(
    s: float,
    T: float,
    r: float,
    sigma: float
):
    """Compute delta_plus."""
    return (np.log(s) + (r + sigma**2) * T) / (sigma * T)


def delta_minus(
    s: float,
    T: float,
    r: float,
    sigma: float
):
    """Compute delta_minus."""
    return (np.log(s) + (r - sigma**2) * T) / (sigma * T)


def call(
    S: float, 
    K: float,
    T: float,
    r: float,
    sigma: float
):
    """Black-Scholes closed formula for a call option."""
    return S * norm.cdf(delta_plus(s=S/K, T=T, r=r, sigma=sigma)) - np.exp(-r*T) * K * norm.cdf(delta_minus(s=S/K, T=T, r=r, sigma=sigma))


def put(
    S: float, 
    K: float,
    T: float,
    r: float,
    sigma: float
):
    """Black-Scholes closed formula for a put option."""
    return np.exp(-r*T) * K * norm.cdf(-delta_minus(s=S/K, T=T, r=r, sigma=sigma)) - S * norm.cdf(-delta_plus(s=S/K, T=T, r=r, sigma=sigma))


## Knock-Out Barrier

Formules basées sur : https://personal.ntu.edu.sg/nprivault/MA5182/barrier-options.pdf

ÉCRIRE LES PRIX ET DÉCIDER DE SI ON DOIT INCLURE LA DÉMONSTRATION POUR AU MOINS UN PRIX.

In [ ]:
def up_out_call(
    S: float, 
    K: float,
    B: float,
    T: float,
    r: float,
    sigma: float
):
    """Black-Scholes closed formula for an up-and-out call option."""
    # If the price is breaking the barrier already, the value is zero
    if S >= B:
        return 0
    # If the up-and-out barrier is lower than the strike, the price is zero (the payoff can only be zero)
    if B <= K:
        return 0
    else:
        return S * (- norm.cdf(delta_plus(s=S / B, T=T, r=r, sigma=sigma)) - 
        ((B / S) ** (1+2*r/(sigma**2))) * (norm.cdf(delta_plus(s=B**2/(K*S), T=T, r=r, sigma=sigma)) - norm.cdf(delta_plus(s=B / S, T=T, r=r, sigma=sigma)))) - np.exp(-r*T) * K * (
            - norm.cdf(delta_minus(s=S / B, T=T, r=r, sigma=sigma)) - 
        ((S / B) ** (1-2*r/(sigma**2))) * (norm.cdf(delta_minus(s=B**2/(K*S), T=T, r=r, sigma=sigma)) - norm.cdf(delta_minus(s=B / S, T=T, r=r, sigma=sigma)))
        ) + call(S=S, K=K, T=T, r=r, sigma=sigma)


def up_out_put(
    S: float, 
    K: float,
    B: float,
    T: float,
    r: float,
    sigma: float
):
    """Black-Scholes closed formula for an up-and-out put option."""
    # If the price is breaking the barrier already, the value is zero
    if S >= B:
        return 0
    if B <= K:
        return S * (norm.cdf(delta_plus(s=S / B, T=T, r=r, sigma=sigma)) - 1 - (B / S) ** (1+2*r/(sigma**2)) * (norm.cdf(delta_plus(s=B/S, T=T, r=r, sigma=sigma)) - 1)) - K * np.exp(-r*T) * (
        norm.cdf(delta_minus(s=S / B, T=T, r=r, sigma=sigma)) - 1 - (S / B) ** (1-2*r/(sigma**2)) * (norm.cdf(delta_plus(s=B/S, T=T, r=r, sigma=sigma)) - 1)
        )
    else:
        return S * (norm.cdf(delta_plus(s=S / K, T=T, r=r, sigma=sigma)) - 1 - (B / S) ** (1+2*r/(sigma**2)) * (norm.cdf(delta_plus(s=B**2/(K*S), T=T, r=r, sigma=sigma)) - 1)) - K * np.exp(-r*T) * (
        norm.cdf(delta_minus(s=S / K, T=T, r=r, sigma=sigma)) - 1 - (S / B) ** (1-2*r/(sigma**2)) * (norm.cdf(delta_plus(s=B**2/(K*S), T=T, r=r, sigma=sigma)) - 1)
        )


def down_out_call(
    S: float, 
    K: float,
    B: float,
    T: float,
    r: float,
    sigma: float
):
    """Black-Scholes closed formula for an down-and-out call option."""
    # If the price is breaking the barrier already, the value is zero
    if S <= B:
        return 0
    if B <= K:
        return call(S=S, K=K, T=T, r=r, sigma=sigma) - S * (B/S)**(2*r/sigma**2) * call(S=B/S, K=K/B, T=T, r=r, sigma=sigma)
    else:
        return S * norm.cdf(delta_plus(s=S/B, T=T, r=r, sigma=sigma
        )) - K * np.exp(-r*T) * norm.cdf(delta_minus(s=S/B, T=T, r=r, sigma=sigma
        )) - B * (B/S)**(2*r/(sigma**2)) * norm.cdf(delta_plus(s=B/S, T=T, r=r, sigma=sigma
        )) + K * np.exp(-r*T) * (S/B)**(1-2*r/(sigma**2)) * norm.cdf(delta_minus(s=B/S, T=T, r=r, sigma=sigma))


def down_out_put(
    S: float, 
    K: float,
    B: float,
    T: float,
    r: float,
    sigma: float
):
    """Black-Scholes closed formula for an down-and-out put option."""
    # If the price is breaking the barrier already, the value is zero
    if S <= B:
        return 0
    if B >= K:
        return 0
    else:
        return put(S=S, K=K, T=T, r=r, sigma=sigma) + S * norm.cdf(-delta_plus(s=S/B, T=T, r=r, sigma=sigma
        )) - B * (B/S)**(2*r/(sigma**2)) * (norm.cdf(delta_plus(s=B**2/(K*S), T=T, r=r, sigma=sigma)) - norm.cdf(delta_plus(s=B/S, T=T, r=r, sigma=sigma))
        ) - K * np.exp(-r*T) * norm.cdf(-delta_minus(s=S/B, T=T, r=r, sigma=sigma
        )) + K * np.exp(-r*T) * (S/B)**(1-2*r/(sigma**2)) * (norm.cdf(delta_minus(s=B**2/(K*S), T=T, r=r, sigma=sigma)) - norm.cdf(delta_minus(s=B/S, T=T, r=r, sigma=sigma))
        )

In [117]:
S = 80
K = 79
B = 70
T = 100 / 365
r = 0.05
sigma = 0.1

price = up_out_call(S=S, K=K, B=B, T=T, r=r, sigma=sigma)
price

0

## Knock-In Barrier

Formules basées sur https://www.asc.tuwien.ac.at/~juengel/simulations/fincalculator/doc/Barrier.pdf

Je suis un peu étonné parce que les formules comme c_ui = c - c_uo sont valables pour tout K, B (X, H dans le document) mais il garde la disjonction de cas H < B vs H >= B au lieu de simplifier...

In [118]:
def down_in_call(
    S: float, 
    K: float,
    B: float,
    T: float,
    r: float,
    sigma: float
):
    """
    Black-Scholes closed formula for an down-and-in call option.
    c_di = c - c_do
    """
    return call(S=S, K=K, T=T, r=r, sigma=sigma) - down_out_call(S=S, K=K, B=B, T=T, r=r, sigma=sigma)


def down_in_put(
    S: float, 
    K: float,
    B: float,
    T: float,
    r: float,
    sigma: float
):
    """
    Black-Scholes closed formula for an down-and-in put option.
    p_di = p - p_do
    """
    return put(S=S, K=K, T=T, r=r, sigma=sigma) - down_out_put(S=S, K=K, B=B, T=T, r=r, sigma=sigma)


def up_in_call(
    S: float, 
    K: float,
    B: float,
    T: float,
    r: float,
    sigma: float
):
    """
    Black-Scholes closed formula for an up-and-in call option.
    c_ui = c - c_uo
    """
    return call(S=S, K=K, T=T, r=r, sigma=sigma) - up_out_call(S=S, K=K, B=B, T=T, r=r, sigma=sigma)


def up_in_put(
    S: float, 
    K: float,
    B: float,
    T: float,
    r: float,
    sigma: float
):
    """
    Black-Scholes closed formula for an up-and-in call option.
    p_ui = p - p_uo
    """
    return put(S=S, K=K, T=T, r=r, sigma=sigma) - up_out_put(S=S, K=K, B=B, T=T, r=r, sigma=sigma)

In [127]:
S = 60
K = 65
B = 67
T = 100 / 365
r = 0.05
sigma = 0.1

price = up_in_call(S=S, K=K, B=B, T=T, r=r, sigma=sigma)
price

0.008645082249772507

# Draft

TO DO:
- Plot price surfaces and compare to the article.
- Decide what should be included in the report and write the corresponding Latex.

DONE:
- Implementation of all the barrier options closed formulas in the Black-Scholes model.